# Data Cleaning

In [40]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.data_processing import impute_by_grouping, count_list_items

df = pd.read_csv('../data/processed/cars_60k.csv')
cols = df.columns

numeric_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(exclude=['number']).columns

/var/folders/yk/_pxs4k190zv7qyl1d8qf38x00000gn/T/ipykernel_32627/1560343195.py:11: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/cars_60k.csv')


## Handle missing values

In [41]:
def handle_missing(df):
    # drop missing values for columns that cant be filled by some logical value
    df = df.dropna(subset=['prod_year', 'model_id'])
    df = df.dropna(subset=['price'])
    df = df.dropna(subset=['comfort_features'])

    # fill missing values where possible
    df['color_id'] = df['color_id'].fillna(df['color_id'].mode()[0])

    # impute missing values using other cars with same man_id and model_id
    # replace numeric with median, categorical with mode
    impute_config = {
        'engine_volume': 'median',
        'cylinders': 'mode',
        'drive_type_id': 'mode',
        'fuel_type_id': 'mode',
        'gear_type_id': 'mode',
    }

    group_cols = ['man_id', 'model_id']
    df = impute_by_grouping(df, impute_config, group_cols)

    # drop if could not imputed using model
    df = df.dropna(subset=['gear_type_id', 'drive_type_id', 'cylinders', 'engine_volume'])

    # 0 is reasonable here shouldn't have been replaced by nans in the first place
    df['car_run_km'] = df['car_run_km'].fillna(0)
    df['user_type'] = df['user_type'].fillna(0)
    df['vehicle_type'] = df['vehicle_type'].fillna(0)

    return df

# print before cleaning
print(f'\nnumeric columns:\n{numeric_cols}')
print(f'\ncategorical columns:\n{categorical_cols}')

# replace missing values 0 with nan
df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] == 0, np.nan)

print(f'\nmissing values before cleaning:\n{df.isnull().sum()}')

# handle missing values
df = handle_missing(df)

# check there are no more missing values
print(f'\nnull values after cleaning:\n{df.isnull().sum()}')
print(f'\nrows in df after: {len(df)}')


numeric columns:
Index(['car_id', 'prod_year', 'man_id', 'model_id', 'price', 'fuel_type_id',
       'gear_type_id', 'drive_type_id', 'color_id', 'cylinders', 'car_run_km',
       'engine_volume', 'vehicle_type', 'category_id', 'user_type'],
      dtype='object')

categorical columns:
Index(['abs', 'esd', 'el_windows', 'conditioner', 'leather', 'hydraulics',
       'chair_warming', 'climat_control', 'customs_passed', 'tech_inspection',
       'has_turbo', 'right_wheel', 'start_stop', 'back_camera',
       'comfort_features'],
      dtype='object')

missing values before cleaning:
car_id                  0
prod_year             884
man_id                  0
model_id            22379
price               27653
fuel_type_id            7
gear_type_id         2224
drive_type_id       22553
color_id            13706
cylinders           23868
car_run_km           9355
engine_volume       24390
abs                     0
esd                     0
el_windows              0
conditioner           

## Normalize boolean columns

In [42]:
boolean_cols = boolean_cols = [col for col in categorical_cols if col != 'comfort_features']

def normalize_boolean(series):
    return (series.astype(str).str.strip().str.lower()
        .map({
            'true': 1,
            '1': 1,
            'yes': 1,

            'false': 0,
            '0': 0,
            'no': 0,
        })
    )

# normalize all boolean columns
for col in boolean_cols:
    df[col] = normalize_boolean(df[col])

# print to make sure all is either 0 or 1
for col in boolean_cols:
    print(f"\n{df[col].value_counts()}\n")


abs
1    27268
0     4242
Name: count, dtype: int64


esd
1    23904
0     7606
Name: count, dtype: int64


el_windows
1    26907
0     4603
Name: count, dtype: int64


conditioner
1    25885
0     5625
Name: count, dtype: int64


leather
0    28992
1     2518
Name: count, dtype: int64


hydraulics
1    27486
0     4024
Name: count, dtype: int64


chair_warming
1    16459
0    15051
Name: count, dtype: int64


climat_control
1    22440
0     9070
Name: count, dtype: int64


customs_passed
1    20378
0    11132
Name: count, dtype: int64


tech_inspection
1    24810
0     6700
Name: count, dtype: int64


has_turbo
0    26493
1     5017
Name: count, dtype: int64


right_wheel
0    27559
1     3951
Name: count, dtype: int64


start_stop
0    30275
1     1235
Name: count, dtype: int64


back_camera
0    16722
1    14788
Name: count, dtype: int64



## Feature engineering & Outlier handling

In [43]:
df = df.copy()

df['car_age'] = 2025 - df['prod_year'] # could use 2022 for more realistic age here as data is from 2022

df['is_dealer'] = (df['user_type'] == 1).astype(int)

df.rename(columns={'vehicle_type': 'is_spec'}, inplace=True)

df['km_per_year'] = df['car_run_km'] / (df['car_age'] + 1)

safety_features = ['abs', 'esd', 'back_camera']
df['safety_feature_count'] = df[safety_features].sum(axis=1)

df['comfort_feature_count'] = df['comfort_features'].apply(count_list_items)


# if comfort_feature list is empty count true values from this
comfort_features = ['el_windows', 'conditioner', 'leather', 'chair_warming', 'climat_control', 'start_stop']

mask = df['comfort_feature_count'] == 0
df.loc[mask, 'comfort_feature_count'] = df.loc[mask, comfort_features].fillna(0).astype(int).sum(axis=1)

# fix some types
df['car_age'] = df['car_age'].astype(int)
df['comfort_feature_count'] = df['comfort_feature_count'].astype(int)
df['safety_feature_count'] = df['safety_feature_count'].astype(int)
df['cylinders'] = df['cylinders'].astype(int)
df['is_spec'] = df['is_spec'].astype(int)

# convert id from float to int
for col in df.columns:
    if col.endswith('_id'):
        df[col] = df[col].astype(int)

### outlier handling

# age 
df = df[(df['car_age'] >= 0) & (df['car_age'] <= 100)]

# price
p01, p99 = df['price'].quantile([0.01, 0.99])
df['price'] = df['price'].clip(p01, p99)

# car_run
df = df[df['car_run_km'] <= 700_000]

# engine volume
df = df[(df['engine_volume'] >= 500) & (df['engine_volume'] <= 8000)]

# cylinders
df = df[(df['cylinders'] >= 2) & (df['cylinders'] <= 16)]


# final column list
NUMERIC_FEATURES = [
    'car_id',
    'price',
    'car_age',
    'engine_volume',
    'cylinders',
    'car_run_km',
    'safety_feature_count',
    'comfort_feature_count'
]

CATEGORICAL_FEATURES = [
    'man_id',
    'model_id',
    'fuel_type_id',
    'gear_type_id',
    'drive_type_id',
    'color_id',
    'category_id'
]

BOOLEAN_FEATURES = [
    'abs', 'esd', 'el_windows', 'conditioner', 'leather',
    'hydraulics', 'chair_warming', 'climat_control',
    'customs_passed', 'tech_inspection',
    'has_turbo', 'right_wheel',
    'start_stop', 'back_camera',
    'is_dealer', 'is_spec'
]

FINAL_COLS = NUMERIC_FEATURES + CATEGORICAL_FEATURES + BOOLEAN_FEATURES

df_final = df[FINAL_COLS].copy()

assert df_final.isna().sum().sum() == 0
assert (df_final['price'] > 0).all()

df_final.to_csv("../data/processed/cars_clean.csv", index=False)

In [44]:
import json

df_final = pd.read_csv('../data/processed/cars_clean.csv')

# Load JSON Mappings
json_dir1 = '../id_mappings/id_mappings.json'
json_dir2 = '../id_mappings/model_id_mapping.json'

with open(json_dir1, 'r') as f:
    data1 = json.load(f)
with open(json_dir2, 'r') as f:
    data2 = json.load(f)

# Build Dictionaries
category_map = {int(item['category_id']): item['title'] for item in data1['vehicle_category_mappings']}
manufacturer_map = {int(item['man_id']): item['title'].strip() for item in data1['manufacturer_mappings']}
fuel_map = {int(item['fuel_type_id']): item['title'].strip() for item in data1['fuel_type_mappings']}
transmission_map = {int(item['id']): item['title'] for item in data1['transmission_type_mappings']}
color_map = {int(item['color_id']): item['title'] for item in data1['color_mappings']}
drive_map = {int(item['drive_type_id']): item['title'] for item in data1['drive_type_mappings']}
model_map = {int(item['model_id']): item['title'].strip() for item in data2}

# apply Mapping
id_cols = ['category_id', 'man_id', 'gear_type_id', 'color_id', 'drive_type_id', 'model_id']

# Safety: Ensure columns are integers (handling potential NaNs)
for col in id_cols:
    # Coerce errors -> NaN, fill with -1, convert to int
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(-1).astype(int)

# map values
df_final['category_id'] = df_final['category_id'].map(category_map)
df_final['man_id'] = df_final['man_id'].map(manufacturer_map)
df_final['fuel_type_id'] = df_final['fuel_type_id'].map(fuel_map)
df_final['gear_type_id'] = df_final['gear_type_id'].map(transmission_map)
df_final['color_id'] = df_final['color_id'].map(color_map)
df_final['drive_type_id'] = df_final['drive_type_id'].map(drive_map)
df_final['model_id'] = df_final['model_id'].map(model_map)

# drop missing mappings
df_final = df_final.dropna(subset=id_cols)

# Verify
print("Unique Transmissions:", df_final['gear_type_id'].unique())
print("Missing values:", df_final[id_cols].isnull().sum().sum())

# rename id cols
df_final = df_final.rename(columns={
    'category_id': 'category',
    'man_id': 'manufacturer',
    'fuel_type_id': 'fuel_type',
    'gear_type_id': 'gear_type',
    'color_id': 'color',
    'drive_type_id': 'drive_type',
    'model_id': 'model'
})

df_final.to_csv("../data/processed/cars_clean.csv", index=False)

print(df_final.shape)

Unique Transmissions: ['Tiptronic' 'Automatic' 'Variator' 'Manual']
Missing values: 0
(27793, 31)
